* NLP 모델링 작업을 수행하기 위해 데이터를 전처리하는 클래스와 함수를 직접 작성하며 프랑스어를 영어로 번역하는 신경망을 학습

### 준비하기

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 데이터 파일 로드
* 이 번역기 프로젝트의 데이터셋은 수천 개의 영어-프랑스어 번역 쌍으로 이루어진 데이터셋

* SOS_token: “Start Of Sentence" 토큰의 약자입니다. 시퀀스의 시작을 나타내는 데 자주 사용됩니다. 기계 번역이나 대화 시스템의 문장과 같은 시퀀스를 처리할 때 모델에 시퀀스의 시작을 나타내는 토큰이 필요할 수 있으며, SOS_token이 이러한 용도로 사용됩니다.

* EOS_token: “End Of Sentence" 토큰을 의미합니다. 시퀀스의 끝을 나타내는 데 사용됩니다. 시작을 나타내는 토큰이 필요한 것처럼, 특히 출력 시퀀스의 길이가 달라질 수 있는 작업에서는 시퀀스의 끝을 나타내는 토큰도 필요합니다.

* 이러한 토큰은 다양한 길이의 시퀀스를 처리할 때 순환 신경망(RNN)과 트랜스포머 모델에 특히 유용합니다. 모델이 시퀀스의 시작과 끝을 인식하는 데 도움이 되며, 이는 기계 번역, 텍스트 요약 등과 같은 작업에 매우 중요할 수 있습니다.

In [2]:
# SOS, EOS 토큰 정의
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        """
        어휘를 추적하기 위해 Lang 오브젝트를 초기화
        
        인자:
        name (str): 언어명
        
        속성:
        name (str): 언어명
        word2index (dict): 단어를 해당 인덱스로의 매핑
        word2count (dict): 단어를 해당 단어의 사용 빈도로 매핑
        index2word (dict): 인덱스에서 단어로의 매핑. SOS 및 EOS에 대한 매핑으로 초기화
        n_words (int): SOS 및 EOS를 포함한 어휘의 총 단어 개수
        """
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2

    def addSentence(self, sentence):
        """
        문장의 모든 단어를 어휘에 추가합니다.
        
        인자:
        sentence (str): 단어가 공백으로 구분된 문장
        """
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        """
        어휘에 단어를 추가
        
        인자:
        word (str): 어휘에 추가할 단어
        """
        if word not in self.word2index:
            # 단어가 어휘에 없는 경우 사용 가능한 다음 인덱스로 추가
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            # 단어가 이미 어휘에 있는 경우 개수를 늘립니다.
            self.word2count[word] += 1

* 파일은 모두 유니코드로 되어 있으므로 간소화하기 위해 유니코드 문자를 ASCII로 바꾸고, 모든 문자를 소문자로 만들고, 구두점을 대부분 잘라냅니다.

In [3]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# 소문자, 자르기(trim) 및 문자가 아닌 문자 제거
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip()) # 소문자로 변환
    s = re.sub(r"([.!?])", r" \1", s) 
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

* 데이터 파일을 읽기 위해 파일을 줄(line)로 분할하고 다시 줄을 쌍으로 분할
* 파일은 모두 영어 → 기타 언어이므로 기타 언어 → 영어에서 번역하려면 쌍을 반대로 바꾸기 위해 역방향 플래그를 추가

In [4]:
import os

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # 파일을 읽고 여러 줄로 분할
    lines = open(os.getenv("HOME") + '/aiffel/pytorch_nlp/data/data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # 모든 줄을 쌍으로 분할하고 정규화
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # 리버스(reverse) 쌍, Lang 인스턴스 만들기
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

* 데이터셋을 비교적 짧고 간단한 문장으로만 정리
* 여기서 최대 길이는 10단어(문장 부호 포함)
* '나는' 또는 '그는' 등의 형태로 번역되는 문장으로 필터링 (앞서 교체된 작은 따옴표(')를 고려)

In [5]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

#### 데이터 준비의 전체 프로세스
* 텍스트 파일을 읽고 줄로 분할하고, 줄을 쌍으로 분할
* 텍스트 정규화, 길이 및 내용별로 필터링
* 쌍으로 된 문장에서 단어 목록 만들기

In [6]:
import random

def prepareData(lang1, lang2, reverse=False):
    """
    문장 쌍을 읽고, 필터링하고, 단어 수를 세어 학습용 데이터를 준비합니다.
    
    인자:
    lang1 (str): 소스 언어의 이름
    lang2 (str): 타겟 언어의 이름
    reverse (bool, optional): True이면 소스 언어와 타겟 언어의 순서를 반대로 합니다. 기본값은 False
    
    리턴:
    tuple: 다음을 포함하는 튜플:
        - input_lang (Lang): 소스 언어에 대한 Lang 오브젝트
        - output_lang (Lang): 타겟 언어에 대한 Lang 오브젝트
        - pairs (list of tuples): 문장 쌍의 목록
    """
    # 언어 쌍을 읽고 Lang 오브젝트 만들기
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    
    # 읽은 문장 쌍의 수 출력
    print("Read %s sentence pairs" % len(pairs))
    
    # 특정 기준(예: 길이)에 따라 문장 쌍을 필터링
    pairs = filterPairs(pairs)
    
    # 필터링 후 문장 쌍의 수를 출력
    print("Trimmed to %s sentence pairs" % len(pairs))
    
    # 단어 카운트가 시작되었음을 나타내는 메시지를 출력
    print("Counting words...")
    
    # 쌍에서 각 Lang 오브젝트에 문장을 추가
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    
    # 각 언어에 대해 계산된 단어 수를 출력
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    
    # Lang 오브젝트와 문장 쌍을 리턴
    return input_lang, output_lang, pairs

# 데이터를 준비하고 임의의 문장 쌍을 출력
input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

### Seq2Seq 모델

#### 인코더
* seq2seq 네트워크의 인코더는 입력 문장의 모든 단어에 대해 특정 값을 출력하는 RNN
* 인코더는 모든 입력 단어에 대해 벡터와 숨겨진 상태를 출력하고 다음 입력 단어에 숨겨진 상태를 사용
* GRU는 LSTM을 개량한 구조로 LSTM에 비해 전체적으로 파라미터가 작고 계산량이 적음

In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

#### 디코더
* 디코더는 인코더 출력 벡터를 가져와 번역을 생성하기 위해 단어 시퀀스를 출력하는 또 다른 RNN
* 초기 입력 토큰은 문자열 시작 SOS 토큰
* 첫 번째 숨겨진 상태는 컨텍스트 벡터(인코더의 마지막 숨겨진 상태)

In [8]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: 다음 입력으로 타겟을 제공(feed)
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: 자체 예측을 다음 입력으로 사용
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # 기록(history)에서 입력으로 분리

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # 학습 루프의 일관성을 위해 'None'을 반환

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

#### 어텐션 디코더
* 인코더와 디코더 사이에 컨텍스트 벡터만 전달되는 경우, 단일 벡터가 전체 문장을 인코딩해야 하는 부담을 안게 됨
* 어텐션은 디코더 네트워크가 디코더 자체 출력 단계마다 인코더 출력의 다른 부분에 "집중"할 수 있도록 함
* 어텐션 가중치 세트를 계산 후 여기에 인코더 출력 벡터를 곱하여 가중치 조합을 생성
* 결과(코드에서는 attn_applied라고 함)에는 입력 시퀀스의 특정 부분에 대한 정보가 포함되어 있어야 함
* 어텐션 가중치 계산은 디코더의 입력과 숨겨진 상태를 입력으로 사용하는 또 다른 피드포워드 레이어 attn으로 수행
* 최대 길이의 문장은 모든 어텐션 가중치를 사용하지만, 짧은 문장은 처음 몇 개만 사용

* 바흐다나우 어텐션을 사용
* 바흐다나우(Bahdanau) 어텐션은 시퀀스 간 모델, 특히 신경 기계 번역 작업에서 일반적으로 사용되는 어텐션 메커니즘
* 학습된 정렬 모델을 사용하여 인코더와 디코더 숨겨진 상태 간의 어텐션 점수를 계산
* 이 모델은 피드포워드 신경망을 사용하여 정렬 점수를 계산

In [9]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: 다음 입력으로 타겟을 공급(feed)
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: 자체 예측을 다음 입력으로 사용
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # 기록에서 입력으로 분리

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

#### 학습 데이터 준비하기
* 학습을 위해서는 각 쌍에 대해 입력 텐서(입력 문장에 있는 단어의 인덱스)와 목표 텐서(목표 문장에 있는 단어의 인덱스)가 필요
* 이러한 벡터를 생성하는 동안 두 시퀀스 모두에 EOS 토큰을 추가

In [10]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

#### 모델 학습
* 인코더를 통해 입력 문장을 실행하고 모든 출력값과 최근 은닉 상태(hidden state)를 추적
* 디코더에 첫 번째 입력으로 <SOS> 토큰이 주어지고 인코더의 마지막 숨겨진 상태가 첫 번째 숨겨진 상태로 주어짐
* Teacher forcing 개념 사용
* "Teacher forcing"는 디코더의 추측을 다음 입력으로 사용하는 대신 실제 목표 출력을 각 다음 입력으로 사용하는 개념
* Teacher forcing을 사용하면 더 빠르게 수렴할 수 있지만, 학습된 네트워크가 잘못 사용되면 불안정성을 보일 수 있음

In [11]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

* 현재 시간과 진행률(%)를 기준으로 경과된 시간과 남은 예상 시간을 인쇄하는 헬퍼(helper) 함수

In [12]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

#### 전체 학습 과정
* 타이머 시작
* 옵티마이저 및 기준 초기화하기
* 학습 쌍 세트 생성
* 플로팅(plotting)을 위해 빈(empty) 손실값 배열을 시작
* 그런 다음 학습을 여러 번 호출하고 진행 상황(예제의 %, 지금까지의 시간, 예상 시간)과 평균 손실을 가끔씩 출력

In [13]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # 모든 print_every마다 리셋
    plot_loss_total = 0  # 모든 plot_every마다 리셋

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

#### 결과 플로팅을 이용해 시각화하기
* 플로팅은 학습 중에 저장된 손실 값 plot_losses 배열을 사용하여 matplotlib로 수행

In [14]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 이 로케이터는 일정한 간격으로 틱을 배치
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

### 평가
* 단어를 예측할 때마다 출력 문자열에 추가
*  EOS 토큰을 예측하면 거기서 멈춤
* 나중에 표시할 수 있도록 디코더의 주의 출력을 저장

In [15]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

* 학습 데이터셋에서 임의의 문장을 평가하고 입력값, 타겟값 및 출력값을 출력하여 주관적인 품질 판단을 내릴 수 있음

In [17]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

### 훈련 및 평가

In [ ]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

In [ ]:
# 드롭아웃 레이어를 평가 모드로 설정
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

#### 어텐션 시각화
* 어텐션 메커니즘의 유용한 특성은 고도로 해석 가능한 출력

In [ ]:
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])


evaluateAndShowAttention('il n est pas aussi grand que son pere')

evaluateAndShowAttention('je suis trop fatigue pour conduire')

evaluateAndShowAttention('je suis desole si c est une question idiote')

evaluateAndShowAttention('je suis reellement fiere de vous')